In [128]:
# Data Preprocessing
import pandas as pd
import numpy as np
import pickle

In [129]:
deliveries_data=pd.read_csv('deliveries.csv')
matches_data=pd.read_csv('matches.csv')

In [130]:
deliveries_data.head(2)

,match_id,inning,batting_team,bowling_team,over,ball,batter,bowler,non_striker,batsman_runs,extra_runs,total_runs,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder
0,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,1,SC Ganguly,P Kumar,BB McCullum,0,1,1,legbyes,0,NaN,NaN,NaN
1,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,2,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN


In [131]:
matches_data.head(2)

,id,season,city,date,match_type,player_of_match,venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,target_runs,target_overs,super_over,method,umpire1,umpire2
0,335982,2007/08,Bangalore,2008-04-18,League,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen
1,335983,2007/08,Chandigarh,2008-04-19,League,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,241.0,20.0,N,NaN,MR Benson,SL Shastri


In [132]:
matches_data=matches_data.rename(columns={'id':'match_id'})
matches_data.head(3)

,match_id,season,city,date,match_type,player_of_match,venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,target_runs,target_overs,super_over,method,umpire1,umpire2
0,335982,2007/08,Bangalore,2008-04-18,League,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen
1,335983,2007/08,Chandigarh,2008-04-19,League,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,241.0,20.0,N,NaN,MR Benson,SL Shastri
2,335984,2007/08,Delhi,2008-04-19,League,MF Maharoof,Feroz Shah Kotla,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,bat,Delhi Daredevils,wickets,9.0,130.0,20.0,N,NaN,Aleem Dar,GA Pratapkumar


In [133]:
cricket_data=pd.merge(deliveries_data,matches_data,on='match_id',how='left')

In [134]:
cricket_data.head(2)

,match_id,inning,batting_team,bowling_team,over,ball,batter,bowler,non_striker,batsman_runs,...,toss_decision,winner,result,result_margin,target_runs,target_overs,super_over,method,umpire1,umpire2
0,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,1,SC Ganguly,P Kumar,BB McCullum,0,...,field,Kolkata Knight Riders,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen
1,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,2,BB McCullum,P Kumar,SC Ganguly,0,...,field,Kolkata Knight Riders,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen


In [135]:
cricket_data['ball_count']=cricket_data['over']*6 + cricket_data['ball']
cricket_data.sort_values(by=['match_id','inning','ball_count'],inplace=True)

In [136]:
cricket_data.reset_index(drop=True,inplace=True)

In [137]:
cricket_data.groupby(['match_id','inning'])


In [138]:
cricket_data['total_runs'].head(3)

0    1
1    0
2    1
Name: total_runs, dtype: int64

In [145]:
cricket_data['score_last_30_balls']=cricket_data.groupby(['match_id','inning'])['total_runs'].transform(
    lambda x:x.rolling(window=30,min_periods=1).sum())

cricket_data['wickets_last_30_balls']=cricket_data.groupby(['match_id','inning'])['is_wicket'].transform(
    lambda x:x.rolling(window=30,min_periods=1).sum())

In [146]:
cricket_data['cumulative_score'] = cricket_data.groupby(['match_id', 'inning'])['total_runs'].cumsum()

In [141]:
mask=cricket_data['over']< 5
mask

0          True
1          True
2          True
3          True
4          True
          ...  
260915    False
260916    False
260917    False
260918    False
260919    False
Name: over, Length: 260920, dtype: bool

In [142]:
cricket_data.loc[mask,'score_last_30_balls']=cricket_data.loc[mask].groupby(['match_id','inning'])['total_runs'].cumsum()
cricket_data.loc[mask,'wickets_last_30_balls']=cricket_data.loc[mask].groupby(['match_id','inning'])['is_wicket'].cumsum()

In [147]:
cricket_data['cumulative_wickets'] = cricket_data.groupby(['match_id', 'inning'])['is_wicket'].cumsum()

In [148]:
cricket_data['cumulative_score'] = cricket_data.groupby(['match_id', 'inning'])['total_runs'].cumsum()

In [149]:
final_scores=cricket_data.groupby(['match_id','inning'])['cumulative_score'].max().reset_index()
final_scores.rename(columns={'cumulative_score':'total_score'},inplace=True)
cricket_data=pd.merge(cricket_data,final_scores,on=['match_id','inning'],how='left')

In [150]:
cricket_data.columns

Index(['match_id', 'inning', 'batting_team', 'bowling_team', 'over', 'ball',
       'batter', 'bowler', 'non_striker', 'batsman_runs', 'extra_runs',
       'total_runs', 'extras_type', 'is_wicket', 'player_dismissed',
       'dismissal_kind', 'fielder', 'season', 'city', 'date', 'match_type',
       'player_of_match', 'venue', 'team1', 'team2', 'toss_winner',
       'toss_decision', 'winner', 'result', 'result_margin', 'target_runs',
       'target_overs', 'super_over', 'method', 'umpire1', 'umpire2',
       'ball_count', 'score_last_30_balls', 'wickets_last_30_balls',
       'cumulative_wickets', 'cumulative_score', 'total_score'],
      dtype='object')

In [151]:
cricket_data.shape

(260920, 42)

In [152]:
#Removing Unwanted columns
columns= ['fielder','dismissal_kind','player_dismissed','is_wicket','non_striker','extras_type','bowler','batter','match_id','batsman_runs', 'extra_runs',
'extras_type','is_wicket', 'player_dismissed','dismissal_kind', 'fielder', 'season','toss_winner','result','super_over', 'method', 'umpire1', 'umpire2',
'target_overs','venue','player_of_match','team1', 'team2','result_margin', 'target_runs','total_runs','match_type','ball_count','winner','ball']
cricket_data.drop(columns=columns, axis=1, inplace=True)

In [153]:
cricket_data.isnull().sum()

inning                       0
batting_team                 0
bowling_team                 0
over                         0
city                     12397
date                         0
toss_decision                0
score_last_30_balls          0
wickets_last_30_balls        0
cumulative_wickets           0
cumulative_score             0
total_score                  0
dtype: int64

In [154]:
cricket_data.dropna(inplace=True)

In [155]:
cricket_data.head(2)

,inning,batting_team,bowling_team,over,city,date,toss_decision,score_last_30_balls,wickets_last_30_balls,cumulative_wickets,cumulative_score,total_score
0,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,Bangalore,2008-04-18,field,1.0,0.0,0,1,222
1,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,Bangalore,2008-04-18,field,1.0,0.0,0,1,222


In [156]:
cricket_data['batting_team'].unique()

array(['Kolkata Knight Riders', 'Royal Challengers Bangalore',
       'Chennai Super Kings', 'Kings XI Punjab', 'Rajasthan Royals',
       'Delhi Daredevils', 'Mumbai Indians', 'Deccan Chargers',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Sunrisers Hyderabad',
       'Rising Pune Supergiants', 'Gujarat Lions',
       'Rising Pune Supergiant', 'Delhi Capitals', 'Punjab Kings',
       'Lucknow Super Giants', 'Gujarat Titans',
       'Royal Challengers Bengaluru'], dtype=object)

In [157]:
#Replacing some of the team names with their latest names
cricket_data['batting_team'] = cricket_data['batting_team'].replace({
    'Kings XI Punjab': 'Punjab Kings',
    'Delhi Daredevils': 'Delhi Capitals'
})

cricket_data['bowling_team'] = cricket_data['bowling_team'].replace({
    'Kings XI Punjab': 'Punjab Kings',
    'Delhi Daredevils': 'Delhi Capitals'
})

In [158]:
#Updating city names with their latest names
cricket_data['city'] = cricket_data['city'].replace({
    'Bangalore': 'Bengaluru'
})

In [159]:
#Keeping the teams who are playing consistently
consistent_teams = ['Kolkata Knight Riders', 'Chennai Super Kings', 'Rajasthan Royals',
                    'Mumbai Indians', 'Punjab Kings', 'Royal Challengers Bangalore',
                    'Delhi Capitals', 'Sunrisers Hyderabad','Gujarat Titans','Lucknow Super Giants']

In [160]:
cricket_data['city'].unique()

array(['Bengaluru', 'Chandigarh', 'Delhi', 'Mumbai', 'Kolkata', 'Jaipur',
       'Hyderabad', 'Chennai', 'Cape Town', 'Port Elizabeth', 'Durban',
       'Centurion', 'East London', 'Johannesburg', 'Kimberley',
       'Bloemfontein', 'Ahmedabad', 'Cuttack', 'Nagpur', 'Dharamsala',
       'Kochi', 'Indore', 'Visakhapatnam', 'Pune', 'Raipur', 'Ranchi',
       'Abu Dhabi', 'Rajkot', 'Kanpur', 'Sharjah', 'Dubai', 'Navi Mumbai',
       'Lucknow', 'Guwahati', 'Mohali'], dtype=object)

In [161]:
#Keeping only consistent cities
consistent_cities= ['Chandigarh', 'Delhi', 'Mumbai', 'Kolkata', 'Jaipur',
       'Hyderabad', 'Chennai',  'Ahmedabad', 'Cuttack', 'Nagpur', 'Dharamsala',
       'Kochi', 'Indore', 'Visakhapatnam', 'Pune', 'Raipur', 'Ranchi',
        'Rajkot', 'Kanpur', 'Bengaluru', 
       'Navi Mumbai', 'Lucknow', 'Guwahati', 'Mohali']

In [162]:
cricket_data = cricket_data[(cricket_data['batting_team'].isin(consistent_teams)) & (cricket_data['bowling_team'].isin(consistent_teams)) &
 (cricket_data['city'].isin(consistent_cities))]

In [163]:
#coverting date column from string to datetime object 
from datetime import datetime
cricket_data['date']=cricket_data['date'].apply(lambda x:datetime.strptime(x,'%Y-%m-%d'))

In [164]:
#Feature Engineering
ipl_data=pd.get_dummies(cricket_data, columns=['batting_team','bowling_team','toss_decision','city'],dtype=int)

In [165]:
ipl_data.columns

Index(['inning', 'over', 'date', 'score_last_30_balls',
       'wickets_last_30_balls', 'cumulative_wickets', 'cumulative_score',
       'total_score', 'batting_team_Chennai Super Kings',
       'batting_team_Delhi Capitals', 'batting_team_Gujarat Titans',
       'batting_team_Kolkata Knight Riders',
       'batting_team_Lucknow Super Giants', 'batting_team_Mumbai Indians',
       'batting_team_Punjab Kings', 'batting_team_Rajasthan Royals',
       'batting_team_Royal Challengers Bangalore',
       'batting_team_Sunrisers Hyderabad', 'bowling_team_Chennai Super Kings',
       'bowling_team_Delhi Capitals', 'bowling_team_Gujarat Titans',
       'bowling_team_Kolkata Knight Riders',
       'bowling_team_Lucknow Super Giants', 'bowling_team_Mumbai Indians',
       'bowling_team_Punjab Kings', 'bowling_team_Rajasthan Royals',
       'bowling_team_Royal Challengers Bangalore',
       'bowling_team_Sunrisers Hyderabad', 'toss_decision_bat',
       'toss_decision_field', 'city_Ahmedabad', 'ci

In [166]:
# Rearranging the columns accordingly
ipl_data = ipl_data[['inning','date', 'batting_team_Chennai Super Kings', 'batting_team_Delhi Capitals',
       'batting_team_Gujarat Titans', 'batting_team_Kolkata Knight Riders',
       'batting_team_Lucknow Super Giants', 'batting_team_Mumbai Indians',
       'batting_team_Punjab Kings', 'batting_team_Rajasthan Royals',
       'batting_team_Royal Challengers Bangalore',
       'batting_team_Sunrisers Hyderabad', 'bowling_team_Chennai Super Kings',
       'bowling_team_Delhi Capitals', 'bowling_team_Gujarat Titans',
       'bowling_team_Kolkata Knight Riders',
       'bowling_team_Lucknow Super Giants', 'bowling_team_Mumbai Indians',
       'bowling_team_Punjab Kings', 'bowling_team_Rajasthan Royals',
       'bowling_team_Royal Challengers Bangalore',
       'bowling_team_Sunrisers Hyderabad',  'toss_decision_bat',
       'toss_decision_field', 'city_Ahmedabad',
       'city_Bengaluru', 'city_Chandigarh', 'city_Chennai', 'city_Cuttack',
       'city_Delhi', 'city_Dharamsala', 'city_Guwahati', 'city_Hyderabad',
       'city_Indore', 'city_Jaipur', 'city_Kolkata', 'city_Lucknow',
       'city_Mohali', 'city_Mumbai', 'city_Navi Mumbai', 'city_Pune',
       'city_Raipur', 'city_Ranchi', 'city_Visakhapatnam','over',  'score_last_30_balls',
       'wickets_last_30_balls', 'cumulative_score','cumulative_wickets', 'total_score',
       ]]

In [167]:
#Splitting the data
x_train=ipl_data.drop(labels='total_score',axis=1)[ipl_data['date'].dt.year<=2022]
x_test=ipl_data.drop(labels='total_score',axis=1)[ipl_data['date'].dt.year>2022]

In [168]:
y_train=ipl_data[ipl_data['date'].dt.year<=2022]['total_score'].values
y_test=ipl_data[ipl_data['date'].dt.year>2022]['total_score'].values

In [169]:
#Removing date column
x_train.drop(labels='date',axis=1,inplace=True)
x_test.drop(labels='date',axis=1,inplace=True)

In [174]:
#Algorithm Selection
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor

In [175]:
models = {
    'LinearRegression': LinearRegression(),
    'Lasso': Lasso(),
    'Ridge': Ridge(),
    'RandomForestRegressor': RandomForestRegressor(),
    'DecisionTreeRegressor': DecisionTreeRegressor(),
    'GradientBoostingRegressor': GradientBoostingRegressor(),
    'XGBRegressor': XGBRegressor(),
    'AdaBoostRegressor': AdaBoostRegressor()
}

In [177]:
results={}
for name,model in models.items():
    scores=cross_val_score(model,x_train,y_train,scoring='neg_mean_squared_error',cv=10)
    results[name]=np.mean(scores)

for name,score in results.items():
    print(f'{name}: Mean MSE: {-score:.4f}')

LinearRegression: Mean MSE: 51474532228290085650432.0000
Lasso: Mean MSE: 547.4876
Ridge: Mean MSE: 543.3325
RandomForestRegressor: Mean MSE: 732.7688
DecisionTreeRegressor: Mean MSE: 1085.7944
GradientBoostingRegressor: Mean MSE: 534.2158
XGBRegressor: Mean MSE: 659.7943
AdaBoostRegressor: Mean MSE: 691.5038


In [180]:
results.get

<function dict.get(key, default=None, /)>

In [181]:
best_model=max(results,key=results.get) #returns the key associated with the maximum value
print(f'Best model is {best_model} with MSE {results[best_model]:.4f}')


Best model is GradientBoostingRegressor with MSE -534.2158


In [182]:
#After retrieving the best model, train the model on entire training dataset
final_model=models[best_model]
final_model.fit(x_train,y_train)

GradientBoostingRegressor()

In [183]:
#Save the model to pickle file
with open('best_model.pkl','wb') as file:
    pickle.dump(final_model,file)